<a href="https://colab.research.google.com/github/Anette-99/API/blob/master/Pipeline_Clean_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se comienza a instalar la libreria de pySpark

In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 8.4 kB/s 
     |████████████████████████████████| 198 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=778c1d1f3dda163b42f7cc055531489b6f9c747a950dd7d2de3c813a8e385d2f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Leer el csv.
Crear un objeto de tipo DataFrameReader usando nuestra propiedad spark.read 
Creamos un objeto de lectura especificando los Headers.

Importamos SparkSession.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

prices = spark.read.csv("prices.csv")
prices.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|      _c0|        _c1|         _c2|    _c3|  _c4|     _c5|     _c6|       _c7|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|03/02/2019|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|28/01/2019|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|   30.00|06/02/2019|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|15/02/2019|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|07/02/2019|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|01/02/2019|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|01/02/2019|
+---------+-----------+------------+-------+-----+--------+--------+----------+



Para poder especificar los nombres de columnas debemos de colocar el siguiente codigo.



In [3]:
prices = spark.read.options(header="true").csv("prices.csv")
prices.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|03/02/2019|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|28/01/2019|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|   30.00|06/02/2019|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|15/02/2019|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|07/02/2019|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|01/02/2019|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|01/02/2019|
+---------+-----------+------------+-------+-----+--------+--------+----------+



Ahora vamos a mostrar los tipos de datos que tiene el Dataset. Lo que debemos de hacer prmero es importar "pprint", lo que nos indicara que tipo de datos hay en el csv.

In [6]:
from pprint import pprint
pprint(prices.dtypes)

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'string'),
 ('currency', 'string'),
 ('quantity', 'string'),
 ('date', 'string')]


Definir el esquema de tipos de datos adecuado.

In [9]:
from pyspark.sql.types import *
schema = StructType([StructField("store", StringType(), nullable=False), StructField("countrycode", StringType(), nullable=False), StructField("brand", StringType(), nullable=False), StructField("model", StringType(), nullable=False), StructField("price", FloatType(), nullable=False), StructField("currency", StringType(), nullable=True), StructField("quantity", IntegerType(), nullable=True), StructField("date", DateType(), nullable=False)])
prices = spark.read.schema(schema).options(header="true").csv("prices.csv")
pprint(prices.dtypes)
prices.show()

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'float'),
 ('currency', 'string'),
 ('quantity', 'int'),
 ('date', 'date')]
+---------+-----------+------------+-------+-----+--------+--------+----+
|    store|countrycode|       brand|  model|price|currency|quantity|date|
+---------+-----------+------------+-------+-----+--------+--------+----+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|null|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|null|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|    null|null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|null|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|null|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|null|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|null|
+---------+-----------+------------+-------+-----+--------+--------+----+



Eliminar las filas erróneas

In [11]:
pricesRowRemoved = spark.read.schema(schema).options(header="true", mode = "DROPMALFORMED").csv("prices.csv")
pricesRowRemoved.show()

+-----+-----------+-----+-----+-----+--------+--------+----+
|store|countrycode|brand|model|price|currency|quantity|date|
+-----+-----------+-----+-----+-----+--------+--------+----+
+-----+-----------+-----+-----+-----+--------+--------+----+



Este codigo va despues de nuestro esquema, para corregir y visualizar la fecha correctamente.

**Podemos visualizar que de este modo estamos limpiando el Datset**

In [12]:
pricesRowRemoved = spark.read.schema(schema).options(header="true", mode = "DROPMALFORMED", dateFormat = "dd/MM/yyyy").csv("prices.csv")
pprint(pricesRowRemoved.dtypes)
pricesRowRemoved.show()

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'float'),
 ('currency', 'string'),
 ('quantity', 'int'),
 ('date', 'date')]
+--------+-----------+------------+-------+-----+--------+--------+----------+
|   store|countrycode|       brand|  model|price|currency|quantity|      date|
+--------+-----------+------------+-------+-----+--------+--------+----------+
|    Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|2019-02-03|
|Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|   Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|      DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|2019-02-01|
|      DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+--------+-----------+------------+-------+-----+--------+--------+----------+



Rellenar valores vacios.

Con la palabra reservada ***fillna*** rellenamos las celdas que aparecen vacias con un valor.

In [15]:
prices3 = spark.read.schema(schema).options(header="true", dateFormat = "dd/MM/yyyy").csv("prices3.csv")
prices3.show()
prices3.fillna(40, subset=["quantity"]).show()
# prices3.fillna({"quantity":40, "date":01/01/1998}).show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|    null|2019-02-03|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|2019-02-06|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|    null|2019-02-15|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|    null|2019-02-01|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|    null|2019-02-01|
+---------+-----------+------------+-------+-----+--------+--------+----------+

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  m

Utilizar condiciones para afectar valores especificos.

**Aqui afectaremos la fecha:**

In [16]:
prices4 = spark.read.schema(schema).options(header="true", dateFormat = "dd/MM/yyyy").csv("prices4.csv")
prices4.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months| 6.87|     EUR|      40|2025-02-03|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|      null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|2025-02-01|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+---------+-----------+------------+-------+-----+--------+--------+----------+



Aqui le vamos a incrementar 1 año: Utilizando el **date.today** significa que estamos por utilizar la fecha de hoy.

In [17]:
from pyspark.sql.functions import col, when
from datetime import date
one_year_from_now = date.today().replace(year=date.today().year + 1)
new_frame = prices4.withColumn("date", when(col("date")> one_year_from_now, None).otherwise(col("date")))
new_frame.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months| 6.87|     EUR|      40|      null|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|      null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|      null|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+---------+-----------+------------+-------+-----+--------+--------+----------+



**Nota:** Este lo podemos utilizar para verificar las fechas mayores a la actual.